In [6]:
pip install numpy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
from datetime import datetime
import pandas as pd
import numpy as np
import datetime
import os
dfgrau = pd.read_csv('../EPSGRAU/eseptiembre2024.csv', sep=',', encoding='ISO-8859-1')
dfgrau = dfgrau.drop('fecha_dataclie', axis=1) 
dfgrau = dfgrau.drop('o_cant_periodos_facturados', axis=1)
#Remplazar codigo de zonal por el nombre que les pertenece
def asignar_valorzonal(row):
    if pd.isna(row['v_id_provincia']):
        return 'Identificarzonal' # Si v_id_provincia es NaN
    elif row['v_id_provincia'] ==1:
        return 'PIURA'
    elif row['v_id_provincia']==4:
        return 'CHULUCANA'
    elif row['v_id_provincia']==5:
        return 'PAITA'
    elif row['v_id_provincia']==6:
        return 'SULLANA'
    elif row['v_id_provincia']==7:
        return 'TALARA'
    else:
        return 'No tiene zonal'

# Aplicar la función de reemplazo de zonal
dfgrau['v_id_provincia'] = dfgrau.apply(asignar_valorzonal, axis=1)

#Reempla
def asignar_valorlocalidad(row):
    if pd.isna((row['v_id_provincia']) or (row['v_id_distrito'])):
        return 'Identificarzonal' # Si CAMPO1 es NaN, CAMPO2 toma el valor de CAMPO3
    elif ((row['v_id_provincia']=='PIURA') & (row['v_id_distrito'] ==1)):
        return 'PIURA'
    elif ((row['v_id_provincia']=='PIURA') & (row['v_id_distrito'] ==2)):
        return 'LAS LOMAS'
    elif ((row['v_id_provincia']=='PIURA') & (row['v_id_distrito'] ==4)):
        return 'CASTILLA'
    elif ((row['v_id_provincia']=='PIURA') & (row['v_id_distrito'] ==5)):
        return 'CATACAOS'
    elif ((row['v_id_provincia']=='TALARA') & (row['v_id_distrito'] ==1)):
        return 'TALARA'
    elif ((row['v_id_provincia']=='TALARA') & (row['v_id_distrito'] ==2)):
        return 'EL ALTO'
    elif ((row['v_id_provincia']=='TALARA') & (row['v_id_distrito'] ==5)):
        return 'LA BREA'
    elif ((row['v_id_provincia']=='TALARA') & (row['v_id_distrito'] ==5)):
        return 'LOS ORGANOS'
    elif ((row['v_id_provincia']=='TALARA') & (row['v_id_distrito'] ==6)):
        return 'MANCORA'
    else:
        return 'No tiene zonal'

# Aplicar la función al DataFrame fila por fila
dfgrau['v_id_distrito'] = dfgrau.apply(asignar_valorlocalidad, axis=1)
valorgrau=21
dfgrau.insert(0,'codigoeps',valorgrau)

dfepsel= pd.read_csv('../CHICLAYO/epselset2024.csv', sep=',', encoding='ISO-8859-1')
valorepsel=9
dfepsel.insert(0,'codigoeps',valorepsel)
dfepsel=dfepsel.drop('o_cant_per', axis=1)
#dfepsel['v_cobranza']=dfepsel['pago_mes']
#dfepsel=dfepsel.drop('pago_mes', axis=1)
dftumb = pd.read_csv('../TUMBES/esetiembre2024.csv', sep=';', encoding='ISO-8859-1')
valortumb=19
dftumb.insert(0,'codigoeps',valortumb)

dfica = pd.read_csv('../ICA/iseptiembre2024.csv', sep=';', encoding='ISO-8859-1')
#df_g['v_fecha_ult_facturacion'] = pd.to_datetime(df_g['v_fecha_ult_facturacion'], format='%d/%m/%Y')
#df_g['v_fecha'] = df_g['v_fecha_ult_facturacion'].astype(str)

valorepica=12
dfica.insert(0,'codigoeps',valorepica)
dfica = dfica.drop('o_cant_periodos_facturados', axis=1)

nuevos_nombres = ['codigoeps','nombre_eps','v_num_inscripcion','v_id_provincia','v_id_distrito','v_id_sector','v_id_manzana','v_id_ciclo','v_nombre_zona',
'v_nombre_via','v_num_habitantes','v_num_pisos','v_id_estado_predio','v_id_tipo_servicio_t','v_estado_facturacion','v_estado_toma_lecturas','v_desc_grupo',
'v_volumen_promedio','v_id_ruta_lectura','v_num_ruta_reparto','v_vol_fact','v_periodo_facturacion','v_medido','v_promedio','v_asignado',
'v_categoria_principal','v_id_situacion_conagua','v_id_situacion_codesa','v_fecha_conagua','v_fecha_codesa','v_presion_conexion_en_mca','v_id_medidor',
'v_diametro_medidor','v_fecha_instalacion_med','v_estado_medidor','v_fact_mes','v_fecha_primer_facturacion','v_fecha_ult_facturacion',
'v_saldo_cta_cte','v_total_deuda_rec_fact','v_pagado','v_rec_pend','v_num_convenio','v_fecha_convenio','v_ult_fecha_corte','v_ult_fecha_reapertura',
'v_desc_sector_comercial','v_fecha_ult_pago','v_ult_tipo_pago','v_tipo_recaudador','v_recaudador','v_cobranza_mes_1','v_cobranza_mes_2','v_cobranza_mes_3',
'v_fecha_venci_ulti_recibo_emitido','o_cant_recibo_pagados','o_importe_total_pagado']
dfgrau.columns = nuevos_nombres
dfgrau= dfgrau[dfgrau['v_fecha_ult_facturacion'].notna() & (dfgrau['v_fecha_ult_facturacion'] != '')]
dfgrau['v_fecha'] = dfgrau['v_fecha_ult_facturacion'].astype(str)

dfepsel.columns = nuevos_nombres
dfica.columns = nuevos_nombres
dftumb.columns = nuevos_nombres

df_g = pd.concat([dfgrau,dftumb,dfica,dfepsel], ignore_index=True)
df_g = df_g.drop_duplicates(subset='v_num_inscripcion')
df_g = df_g[df_g['v_fecha_ult_facturacion'].notna() & (df_g['v_fecha_ult_facturacion'] != '')]
df_g['v_fecha'] = df_g['v_fecha_ult_facturacion'].astype(str)
df_g['v_fecha_instalacion_med'] = pd.to_datetime(df_g['v_fecha_instalacion_med'], errors='coerce')
df_g['v_fecha_instalacion_med'] = df_g['v_fecha_instalacion_med'].dt.strftime('%d/%m/%Y')
df_g['v_fecha_instalacion_med'] = df_g['v_fecha_instalacion_med'].astype(str)
df_g['fecham'] = pd.to_datetime(df_g['v_fecha_instalacion_med'])
# Extraer el año y mes
df_g['mes_v'] = df_g['fecham'].dt.month
df_g['ano_v'] = df_g['fecham'].dt.year

# Obtener la fecha actual
#fecha_actual = datetime.now()



#df_g = pd.DataFrame(df_g)
df_g.loc[(df_g['v_asignado']>0), ['v_promedio', 'v_medido']] = np.nan
df_g.loc[(df_g['v_medido']>0), ['v_promedio', 'v_asignado']] = np.nan
df_g.loc[(df_g['v_promedio']>0), ['v_medido', 'v_asignado']] = np.nan

df_g.loc[df_g['v_asignado'] == 0, 'v_promedio'] = df_g['v_asignado']
df_g.loc[(df_g['v_asignado'] == 0), 'v_asignado'] = np.nan

df_g.loc[(df_g['v_estado_medidor']!='OPERATIVO') & (df_g['v_promedio']==0), ['v_medido']] = np.nan
df_g.loc[(df_g['v_estado_medidor']=='OPERATIVO') & (df_g['v_promedio']==0), ['v_promedio']] = np.nan

df_g['tconsumo']='Asignado'
df_g.loc[df_g['v_medido'].notna(), ['tconsumo']] = 'Medido'
df_g.loc[df_g['v_promedio'].notna(), ['tconsumo']] = 'Promedio'
df_g.loc[df_g['v_asignado'].notna(), ['tconsumo']] = 'Asignado'

#df_g.update(df_g)
df_g.loc[df_g['v_categoria_principal'].isin(['DOMESTICA 1', 'DOMESTICA 2','DOMESTICA']),'v_categoria_principal'] = 'DOMESTICO'


def asignar_valorfiel(v_pagado):
    # Revisar si CAMPO1 es 0, vacío o NaN
    #if v_cobranza_mes_1 == 0 or pd.isna(v_cobranza_mes_1) or v_cobranza_mes_1 == '':
    if v_pagado == 'NO' or v_pagado == 'N' or pd.isna(v_pagado) or v_pagado == '':
        return 0
    else:
        return 1

# Aplicar la función al DataFrame
df_g['fiel'] = df_g['v_pagado'].apply(asignar_valorfiel)

def asignar_valorconsumo(row):
    if pd.isna(row['v_vol_fact']):
        return row['v_categoria_principal']  # Si CAMPO1 es NaN, CAMPO2 toma el valor de CAMPO3
    elif row['v_vol_fact'] > 99:
        return 'ALTOS CONSUMIDORES'
    elif row['v_vol_fact'] < 6:
        return 'BAJOS CONSUMIDORES'
    else:
        return row['v_categoria_principal']

# Aplicar la función al DataFrame fila por fila
df_g['clasificaconsumo'] = df_g.apply(asignar_valorconsumo, axis=1)

def asignar_valorrangoconsumo(v_vol_fact):
    if pd.isna(v_vol_fact):
        return 'Sin consumo'  # Manejo de NaN si es necesario
    elif v_vol_fact == 0:
        return 'Consumos Ceros'
    elif v_vol_fact <= 5:
        return 'Entre 01 a 5 M3'
    elif v_vol_fact <= 20:
        return 'Entre 06 a 20 M3'
    elif v_vol_fact <= 100:
        return 'Entre 21 a 100 M3'
    else:
        return 'Mayor a 100'

# Aplicar la función al DataFrame
df_g['rangoconsumo'] = df_g['v_vol_fact'].apply(asignar_valorrangoconsumo)

def asignar_rangoreciboDeuda(v_rec_pend):
    if pd.isna(v_rec_pend):
        return 'Sin consumo'  # Manejo de NaN si es necesario
    elif v_rec_pend == 0:
        return 'Deuda 0 Mes'
    elif v_rec_pend <= 2:
        return 'Deuda 01 a 02 Meses'
    elif v_rec_pend <= 6:
        return 'Deuda 03 a 06 Meses'
    elif v_rec_pend <= 12:
        return 'Deuda 07 a 12 M3'
    else:
        return 'Deuda de 12 a mas Meses'
    
# Aplicar la función al DataFrame
df_g['rangorecibosdeuda'] = df_g['v_rec_pend'].apply(asignar_rangoreciboDeuda)

rangorecibosdeuda = df_g.groupby(['codigoeps','v_id_provincia','v_id_distrito','v_id_sector','v_fecha','v_categoria_principal','rangorecibosdeuda']).agg(
conexiones=('codigoeps', 'count'),  
v_total_deuda_rec_fact=('v_total_deuda_rec_fact', 'sum'),
v_saldo_cta_cte=('v_saldo_cta_cte', 'sum')
).reset_index()

resultadoClasificacionConsumo = df_g.groupby(['codigoeps','v_id_provincia','v_id_distrito','v_id_sector','v_fecha','v_categoria_principal','clasificaconsumo','rangoconsumo']).agg(
v_medido=('v_medido', 'sum'),
v_promedio=('v_promedio', 'sum'),
v_asignado=('v_asignado', 'sum'),
conexiones=('codigoeps', 'count'),       
v_fact_mes=('v_fact_mes', 'sum'),
v_total_deuda_rec_fact=('v_total_deuda_rec_fact', 'sum'),
v_cobranza_mes_1=('v_cobranza_mes_1', 'sum'),
pagaron=('fiel', 'sum')    
).reset_index()

#clasificacionConsumo= pd.read_csv('G:/Unidades compartidas/Dirección de Operaciones/SEGUNDOLIVIA/DATALIKE/COMERCIAL/epsConsolidado_2024.csv', sep=';', encoding='ISO-8859-1')

#consolidado = pd.concat([clasificacionConsumo,resultadoClasificacionConsumo], ignore_index=True)

resultadoRangoConsumo = df_g.groupby(['codigoeps','v_id_provincia','v_id_distrito','v_id_sector','v_fecha','v_categoria_principal','tconsumo','rangoconsumo']).agg(
v_medido=('v_medido', 'sum'),
v_promedio=('v_promedio', 'sum'),
v_asignado=('v_asignado', 'sum'),
conexiones=('codigoeps', 'count'),       
v_cobranza_mes_1=('v_cobranza_mes_1', 'sum'),
pagaron=('fiel', 'sum')        
).reset_index()

resultadoImporteTotal = df_g.groupby(['codigoeps','v_id_provincia','v_id_distrito','v_fecha']).agg(
o_importe_total_pagado=('o_importe_total_pagado', 'sum'),      
v_cobranza_mes_1=('v_cobranza_mes_1', 'sum'),
pagaron=('fiel', 'sum')        
).reset_index()


#df_g['v_medido'] = df_g['v_medido'].astype(str)

#df_g['v_medido'].notna()

#filtered_df = df_g[(df_g['v_medido'].astype(float)>= 0)]
#filtro = df_g['codigoeps'] == 19

# Aplica el filtro y muestra los valores de campo1
#resultadoRangoAnoMedidores= df_g[filtro]['v_medido']
#print(resultadoRangoAnoMedidores)

#resultadoRangoAnoMedidores = filtered_df.groupby(['codigoeps','v_id_provincia','v_id_distrito','v_fecha','ano_v','mes_v','v_categoria_principal','rangoconsumo']).agg(

resultadoRangoAnoMedidores = df_g.groupby(['codigoeps','v_id_provincia','v_id_distrito','v_fecha','ano_v','mes_v','v_categoria_principal','rangoconsumo']).agg(
conexiones=('codigoeps', 'count')
).reset_index()

ahora = datetime.datetime.now()
ano = ahora.strftime('%Y%m')  # Captura el día en formato 'año'
dia = ahora.strftime('%Y%m%d')  # Captura el día en formato 'año-mes-día'
hora = ahora.strftime('%H')  # Obtiene la hora actual en formato de dos dígitos
minuto = ahora.strftime('%M')  # Obtiene el minuto actual
segundo = ahora.strftime('%S')  # Obtiene el segundo actual

epsClasConsumo= 'epsClasConsumo'
epsRangoConsumo= 'epsRangoConsumo'
epsRangoAnos= 'epsRangoAnosMedidores'
epsImporteTotal= 'epsImporteTotal'
epsTodo= 'epsTodo'
epsConsolidado= 'epsConsolidado'
epsRangoDeudaRecibo= 'epsRangoDeudaRecibo'
nombre_epsTodo = f"{epsTodo}_{dia}{hora}{minuto}{segundo}.csv"
ruta_epsTodo = os.path.join('../CHICLAYO',nombre_epsTodo)
df_g.to_csv(ruta_epsTodo, index=False, sep=';')
# Construye el nombre del archivo con los parámetros capturados
nombre_epsRangoDeudaRecibo = f"{epsRangoDeudaRecibo}_{dia}{hora}{minuto}{segundo}.csv"
ruta_archivoeRangoDeudaRecibo = os.path.join('G:/Unidades compartidas/Dirección de Operaciones/SEGUNDOLIVIA/DATALIKE/COMERCIAL/RangoDeudaRecibo/',nombre_epsRangoDeudaRecibo)
rangorecibosdeuda.to_csv(ruta_archivoeRangoDeudaRecibo, index=False, sep=';')

nombre_epsClasConsumo = f"{epsClasConsumo}_{dia}{hora}{minuto}{segundo}.csv"
ruta_archivoepsClasConsumo = os.path.join('G:/Unidades compartidas/Dirección de Operaciones/SEGUNDOLIVIA/DATALIKE/COMERCIAL/CobranzaFacturacion/',nombre_epsClasConsumo)
resultadoClasificacionConsumo.to_csv(ruta_archivoepsClasConsumo, index=False, sep=';')

nombre_epsRangoConsumo = f"{epsRangoConsumo}_{dia}{hora}{minuto}{segundo}.csv"
ruta_archivoepsRangoConsumo = os.path.join('G:/Unidades compartidas/Dirección de Operaciones/SEGUNDOLIVIA/DATALIKE/COMERCIAL/RangosConsumos/',nombre_epsRangoConsumo)
resultadoRangoConsumo.to_csv(ruta_archivoepsRangoConsumo, index=False, sep=';')

nombre_epsRangoAnos = f"{epsRangoAnos}_{dia}{hora}{minuto}.csv"
ruta_archivoepsRangoAnos = os.path.join('../CHICLAYO',nombre_epsRangoAnos)
resultadoRangoAnoMedidores.to_csv(ruta_archivoepsRangoAnos, index=False, sep=';')

nombre_epsepsImporteTotal= f"{epsImporteTotal}_{dia}.csv"
ruta_archivoepsImporteTotal = os.path.join('../CHICLAYO',nombre_epsepsImporteTotal)
resultadoImporteTotal.to_csv(ruta_archivoepsImporteTotal, index=False, sep=';')

#nombre_epsConsolidado= f"{epsConsolidado}_{ano}.csv"
#ruta_archivoeepsConsolidado = os.path.join('G:/Unidades compartidas/Dirección de Operaciones/SEGUNDOLIVIA/DATALIKE/COMERCIAL/CobranzaFacturacion/',nombre_epsConsolidado)
#consolidado.to_csv(ruta_archivoeepsConsolidado, index=False, sep=';')

#desde aqui hay que anular por el momento, hasta corregir el formato de fechas
# Variables
#directorio = '../CHICLAYO'
#nombre_archivo = nombre_archivo
# Crear la ruta completa
#ruta_completa = f"{directorio}/{nombre_archivo}"

# Leer el archivo CSV
#df = pd.read_csv(ruta_completa)
#df=pd.read_csv(ruta_completa, sep=';', encoding='ISO-8859-1')
#df['v_fecha'] = pd.to_datetime(df['v_fecha'], format='%d-%m-%Y')
#df['ano']=df['v_fecha']
# Extraer el año y el mes
#df['año'] = df['v_fecha'].dt.year
#df['mes'] = df['v_fecha'].dt.month

#print(f"Archivo guardado como: {ruta_archivo}")

In [6]:
import pandas as pd

# Leer el archivo CSV como texto
with open('Libro2.csv', 'r') as file:
    data = file.read()

# Reemplazar los delimitadores múltiples con una coma
data = data.replace(';', ',')  # Reemplaza punto y coma con coma
data = data.replace('|', ',')  # Reemplaza barra vertical con coma

# Guardar el texto procesado en un nuevo archivo CSV
with open('archivo_procesado.csv', 'w') as file:
    file.write(data)

# Leer el archivo procesado con pandas
df = pd.read_csv('archivo_procesado.csv', sep=',')

print(df)

UnicodeDecodeError: 'charmap' codec can't decode byte 0x8d in position 5800742: character maps to <undefined>

In [71]:
cs = df1[['nombre_eps','v_num_inscripcion','v_id_provincia','v_id_distrito','v_id_sector','v_id_manzana','v_id_ciclo','v_nombre_zona','v_nombre_via']]
cs1 =df2[['nombre_eps','v_num_inscripcion','v_id_provincia','v_id_distrito','v_id_sector','v_id_manzana','v_id_ciclo','v_nombre_zona','v_nombre_via']]
##,'v_id_sector','v_id_manzana','v_id_ciclo','v_nombre_zona','v_nombre_via','v_num_habitantes','v_num_pisos','v_id_estado_predio','v_id_tipo_servicio_t','v_estado_facturacion'

#Jun tamos los 03 dataframe en uno solo, con la funcion concat
df_g = pd.concat([cs,cs1], ignore_index=True)

#Creamos una nueva columna, la llamamos año en el dataframe
#Con la funcion de Python str, substraemos los 04 valores del campo fecha, por ser un campo string
#df_g['año'] = df_g['Fecha'].str[:4]
#Creamos una nueva columna ventas en el dataframe
#El valor de la nueva columna Ventas, sera calculada por dos campos
#Cantidad Vendida * Precio Unitario
#df_g['Ventas']  = df_g['Cantidad Vendida'] * df_g['Precio Unitario']

#Total_Ventas = venta.groupby('AÑO')['Ventas'].sum()

#Vamos a utilizar dos funciones de Python groupby y agg
# Con groupby vamos a agrupar por el campo que seleccionemos
# Con agg, podemos crear varios funciones que se almacenen en una sola variable
#Agrupar por año y las ventas
#resultado = df_g.groupby(['año']).agg(
#   Total_ventas=('Ventas', 'sum')
#).reset_index()
#Total de ventas por categoría y año.
# utilizamos agg para realizar las operaciones como son Cantidad Vendidad, ventas, y promedio de Precio Promedio
#Utilizamos reset_index para creaer los indices del reporte del Dataframe
##resultado2 = df_g.groupby(['año', 'Categoria']).agg(
##          cantidad_ventas=('Cantidad Vendida', 'count'),
    ##            Total_ventas=('Ventas', 'sum'),
    ##promedio_precio_unitario=('Precio Unitario', 'mean')
##).reset_index()
#Promedio de precio unitario por categoría, se hizo por separa el calculo del promedio
# Utilizamos la funcion mean para calcular el promedio
##resultado3 = df_g.groupby(['Categoria']).agg(
    # ##  promedio_precio_unitario=('Precio Unitario', 'mean')
##).reset_index()
# Encontrar el producto más vendido por año
#Agrupamos por año y producto
# sumamos el campo Cantidad Vendidas conla funcion sum
##producto_mas_vendido = df_g.groupby(['año', 'Producto'])['Cantidad Vendida'].sum().reset_index()
#Ordenamos por año y Cantidad Vendida de manera ascendente
##producto_mas_vendido = producto_mas_vendido.sort_values(['año', 'Cantidad Vendida'], ascending=[True, False])
# Eliminar duplicados por el campo año
##producto_mas_vendido = producto_mas_vendido.drop_duplicates(subset=['año'])

# Identificar los productos con disminución de ventas en el segundo año
# Agrupamos por año y producto
# La función unstack() se utiliza para transformar el índice de un DataFrame de nivel de filas a nivel de columnas
# básicamente pivotando el DataFrame, Esta operacion permite reorganizar los datos para un análisis más fácil o para preparar los datos para visualización.
# fillna(0) convierte en 0 cundo encuentra NaN
##ventas_por_producto = df_g.groupby(['año', 'Producto'])['Cantidad Vendida'].sum().unstack().fillna(0)
##print(ventas_por_producto) 
# La función diff() calculamos la diferencia entre las cantidades vendidas por producto.
# Es especialmente útil para calcular cambios entre valores consecutivos en una serie temporal
# La operación iloc < 0 se utiliza para acceder a un valor específico en un DataFrame o 
# Serie utilizando el índice basado en la posición (índice entero) y luego comparar ese valor con 0.
##productos_disminucion = ventas_por_producto.diff().iloc[1] < 0
#convertimos en un lista
##productos_disminucion = productos_disminucion[productos_disminucion].index.tolist()
# Imprimir los resultados
print("Resumen por año:")
#print(df_g1)
print(df_g)




Resumen por año:
            nombre_eps  v_num_inscripcion         v_id_provincia  \
0       EPS GRAU S. A.                282                      1   
1       EPS GRAU S. A.                737                      1   
2       EPS GRAU S. A.               1089                      1   
3       EPS GRAU S. A.             276046                      5   
4       EPS GRAU S. A.            1085547                      4   
...                ...                ...                    ...   
308087     AGUA TUMBES            1284006  CONTRALMIRANTE VILLAR   
308088     AGUA TUMBES            1284007  CONTRALMIRANTE VILLAR   
308089     AGUA TUMBES            1237832  CONTRALMIRANTE VILLAR   
308090     AGUA TUMBES             283689  CONTRALMIRANTE VILLAR   
308091     AGUA TUMBES            1239314  CONTRALMIRANTE VILLAR   

       v_id_distrito  v_id_sector  v_id_manzana  v_id_ciclo  \
0                  5            2           450           1   
1                  5            7       

In [ ]:

print("\nProducto más vendido por año:")
print("\n---")
print(producto_mas_vendido)
print("Promedio de precio unitario por categoría:")
print("\n---")
print(resultado3)
print("\nProductos con disminución de ventas en el segundo año:")
print("\n---")
print(productos_disminucion)


Producto más vendido por año:

---
     año Producto  Cantidad Vendida
12  2021    Leche               647
33  2022      Pan               601
42  2023   Azúcar               495
Promedio de precio unitario por categoría:

---
           Categoria  promedio_precio_unitario
0            Bebidas                 10.145122
1  Frutas y Verduras                 10.835412
2             Granos                 10.761477
3           Limpieza                 10.111895
4            Lácteos                 10.414189
5          Panadería                 10.612951
6          Proteínas                 10.631349
7             Snacks                  9.665484

Productos con disminución de ventas en el segundo año:

---
['Azúcar', 'Café', 'Cereal', 'Cerveza', 'Detergente', 'Frutas', 'Jabón', 'Leche', 'Sal', 'Vino']


In [ ]:
print("\nCalcular estadísticas descriptivas:")
print("\n---")
print(df_g.describe())
#df_g.info()






Calcular estadísticas descriptivas:

---
       Cantidad Vendida  Precio Unitario      Ventas
count        900.000000       900.000000  900.000000
mean          25.611111        10.435022  267.333656
std           14.849182         5.426371  221.178019
min            1.000000         1.040000    2.050000
25%           13.000000         5.785000   88.095000
50%           26.000000        10.330000  206.325000
75%           39.000000        15.105000  400.582500
max           50.000000        19.970000  914.830000


In [72]:
#Guardamos el Dataframe
df_g.to_csv('archivo_generarl.csv', index=False)

In [ ]:
class Producto:
    def __init__(self, id_producto, nombre, categoria_id, precio):
        self.id_producto = id_producto
        self.nombre = nombre
        self.categoria_id = categoria_id
        self.precio = precio

class Venta:
    def __init__(self, id_venta, id_producto, cantidad, fecha):
        self.id_venta = id_venta
        self.id_producto = id_producto
        self.cantidad = cantidad
        self.fecha = fecha

class Categoria:
    def __init__(self, id_categoria, nombre):
        self.id_categoria = id_categoria
        self.nombre = nombre

In [ ]:
class ProductoAlimenticio(Producto):
    # Tambien definimos su propio atributo de la clase hija, como es goles_marcados
    def __init__(self, nombre,categoria_id, precio,):
        # llamamos a la clase padre(super clase) y heredando sus atributos nombre y duracion
        super().__init__(nombre,categoria_id, precio)
    
        
class ProductoNoAlimenticio(Producto):
    # Tambien definimos su propio atributo de la clase hija, como es goles_marcados
    def __init__(self, nombre,categoria_id, precio,):
        # llamamos a la clase padre(super clase) y heredando sus atributos nombre y duracion
        super().__init__(nombre,categoria_id, precio)


In [ ]:


def leer_csv(archivo):
    try:
        # Intenta leer el archivo CSV
        datos = pd.read_csv('archivo_generarl.csv')
        return datos
    except FileNotFoundError:
        print(f"Error: El archivo {archivo} no se encontró.")
    except pd.errors.EmptyDataError:
        print(f"Error: El archivo {archivo} está vacío.")
    except pd.errors.ParserError:
        print(f"Error: Ocurrió un problema al analizar el archivo {archivo}.")
    except Exception as e:
        print(f"Error inesperado al leer el archivo: {e}")

def procesar_datos_csv(datos):
    resultados = []
    try:
        # Supongamos que necesitamos dos columnas específicas: 'columna1' y 'columna2'
        if 'año' not in datos.columns or 'Ventas' not in datos.columns:
            raise KeyError("Faltan las columnas necesarias en los datos.")
        
        for _, fila in datos.iterrows():
            try:
                num1 = float(fila['año'])
                num2 = float(fila['Ventas'])
                resultados.append(num1 + num2)
            except ValueError as e:
                print(f"Error de valor: {e} en la fila {fila}.")
            except KeyError as e:
                print(f"Error: Columna no encontrada {e} en la fila {fila}.")
            except Exception as e:
                print(f"Error inesperado al procesar la fila {fila}: {e}")
                
    except KeyError as e:
        print(f"Error: {e}")
    except Exception as e:
        print(f"Error inesperado al procesar los datos: {e}")
    
    return resultados

def main():
    archivo = 'archivo_generarl.csv'
    
    datos = leer_csv(archivo)
    if datos is not None and not datos.empty:
        resultados = procesar_datos_csv(datos)
        print("Resultados:", resultados)

if __name__ == "__main__":
    main()

Resultados: [2046.6, 2065.75, 2239.65, 2444.8, 2536.14, 2117.04, 2510.84, 2052.55, 2143.8, 2662.41, 2662.2, 2280.05, 2576.0, 2091.23, 2261.61, 2056.15, 2521.66, 2140.4, 2205.3, 2127.1, 2811.8, 2043.09, 2149.92, 2751.85, 2076.88, 2152.44, 2247.6, 2361.52, 2045.1, 2378.0, 2462.68, 2078.24, 2279.04, 2078.1, 2227.32, 2354.84, 2270.59, 2589.58, 2254.6, 2179.2, 2347.6, 2415.64, 2507.33, 2100.16, 2039.4, 2279.9, 2355.92, 2647.85, 2095.7, 2066.45, 2873.46, 2095.0, 2305.8, 2151.68, 2494.14, 2051.32, 2684.8, 2238.79, 2028.63, 2203.59, 2351.55, 2462.8, 2212.0, 2112.52, 2436.2799999999997, 2070.4, 2565.62, 2783.19, 2100.35, 2100.52, 2642.81, 2317.52, 2709.0, 2316.51, 2148.84, 2726.76, 2063.75, 2286.16, 2127.06, 2631.94, 2532.5, 2155.73, 2720.46, 2130.4, 2345.6, 2277.08, 2463.34, 2226.04, 2303.88, 2670.29, 2223.32, 2101.72, 2133.5, 2037.2, 2680.04, 2513.56, 2239.32, 2223.75, 2465.24, 2309.9, 2166.28, 2138.3, 2077.55, 2127.19, 2039.4, 2124.36, 2041.88, 2151.44, 2458.5, 2226.68, 2081.08, 2154.3, 2522